In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans

PATH_MELANOMA = '../DermMel/test/Melanoma'
PATH_NOTMELANOMA = '../DermMel/test/NotMelanoma'

img = cv2.imread(PATH_MELANOMA + '/AUG_0_1467.jpeg',0) # leitura em cinza
# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create()

# encontrando os pontos-de-interesse
kp = sift.detect(img,None)

# computando o vetor-de-característica para cada ponto dectado no passo anterior
kp, des = sift.compute(img, kp)
K = 10
kmeans_model = KMeans(n_clusters=K).fit(des) #des são os vetores-de-características

# criando o dicionário de palavras
palavras = np.array(kmeans_model.cluster_centers_)
print(kmeans_model)
#desenhando os pontos na imagem
#img2 = cv2.drawKeypoints(img, kp, None)

#plt.imshow(img2),plt.show()

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


In [ ]:
def gravar_arquivo_arff(base_teste, classes): # 0 para NAO_MELANOMA e 1 para MELANOMA
    tam = len(base_teste[0][0])
    file = open('cance.arff','w') 
 
    file.write('@relation cancer\n') 
    for i in range(tam):
        file.write('@attribute '+ str(i) +' NUMERIC\n') 
    
    file.write('@attribute classes {')
    
    a = set(classes)
    
    for i in a:
        file.write(str(i)+',')
    
    file.write('}')    
    
    for i in range(tam):
         len(set(classes))
    
    file.write('\n@data\n') 

    for item in base_teste:
        for i in range(len(item[0])):
            file.write("%s," % str(item[0][i])) 
        file.write("%s\n" % item[1])    
 
    file.close() 
    print('arquivo gravado')

In [ ]:
def distancia(a, b):
    M = len(a)
    soma = 0
    for i in range(M):
        soma = soma + ((a[i]-b[i])**2)
    return np.sqrt(soma) 

def extrair_caracteristica(img):
    #plt.imshow(img)
    sift = cv2.xfeatures2d.SIFT_create()
    kp = sift.detect(img,None) # encontrando os pontos-de-interesse
    #print('Qtde Pontos Detectados: ', len(kp))
    # computando o vetor-de-característica para cada ponto dectado no passo anterior
    kp, des = sift.compute(img, kp)
    
    palavras = np.array(kmeans_model.cluster_centers_)
    #print(palavras)
    return palavras

In [ ]:
PATH_MELANOMA = '../DermMel/test/Melanoma'
PATH_NOTMELANOMA = '../DermMel/test/NotMelanoma'

img = cv2.imread(PATH_MELANOMA+'/AUG_0_1467.jpeg')

caractericas = extrair_caracteristica(img)

print(caractericas)

In [ ]:
import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from sklearn import metrics

PATH_MELANOMA = '../DermMel/test/Melanoma'
PATH_NOTMELANOMA = '../DermMel/test/NotMelanoma'

files_path_nao_melanoma = [os.path.abspath(x) for x in os.listdir(PATH_MELANOMA) if x.endswith('.jpeg' or '.jpg')]       
files_path_melanoma = [os.path.abspath(x) for x in os.listdir(PATH_NOTMELANOMA) if x.endswith('.jpg' or '.jpeg')] 

all_image_paths = [files_path_nao_melanoma, files_path_melanoma]

base = []
classes = []

#extraindo as características das imagens na base de dados
for i in range(len(all_image_paths)):
    for img_path in all_image_paths[i][:5]:
        imagem = cv2.imread(img_path)
        caracteristica = extrair_caracteristica(imagem)
        classes.append(i)
        base.append((caracteristica, i))
    print('Classe final: '+str(i))

gravar_arquivo_arff(base, classes) 